# 🩺 Clasificación de Melanomas a partir de Imágenes

## 🌑 Introducción
El **melanoma** es un tipo de cáncer de piel que se origina en los **melanocitos**, las células encargadas de producir la melanina, el pigmento que da color a la piel, ojos y cabello.  
Aunque representa un porcentaje pequeño de los cánceres de piel, es el **más agresivo** debido a su gran capacidad para invadir otros tejidos y producir metástasis.  
La detección temprana es fundamental, ya que aumenta considerablemente las posibilidades de un tratamiento exitoso y de supervivencia del paciente.  

---

## 🔎 Metodología clínica: Regla ABCD
En dermatología se utiliza la **regla ABCD** como guía visual para identificar posibles melanomas:  

- **A – Asimetría**: los melanomas suelen tener formas irregulares; un lunar benigno suele ser simétrico.  
- **B – Bordes**: los melanomas presentan bordes difusos, dentados o irregulares, frente a los bordes definidos de los lunares normales.  
- **C – Color**: los melanomas pueden mostrar múltiples tonalidades (negro, marrón, rojo, azul, blanco), mientras que los lunares benignos suelen ser uniformes.  
- **D – Diámetro**: lesiones con un diámetro superior a 6 mm se consideran sospechosas.  

A esta regla a veces se añade la **E – Evolución**, que hace referencia a cambios en el tamaño, forma, color o la aparición de síntomas (picor, sangrado).  

---

## 🎯 Objetivo del trabajo
El objetivo de este proyecto es **desarrollar un modelo de aprendizaje automático capaz de clasificar imágenes de lesiones cutáneas** para distinguir entre melanomas y lesiones benignas.  
Para ello:  
- Se trabajará con un conjunto de imágenes médicas.  
- Se aplicará un flujo de trabajo basado en la preparación y preprocesamiento de datos, entrenamiento de modelos de deep learning y análisis de resultados.  
- Se evaluará el desempeño de los modelos con métricas adecuadas, y se comparará con la metodología clínica tradicional basada en la regla ABCD.  

Este proyecto busca **apoyar el diagnóstico médico mediante técnicas computacionales**, aportando una herramienta complementaria a la práctica clínica.


# IMPORTACIONES Y RUTAS NECESARIAS

In [13]:
# IMPORTACIONES
# Standard library
from pathlib import Path
import shutil

# Third-party
import torch
from PIL import Image, ImageOps
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import functional as TF


# RUTAS

BASE_DIR = Path(".").resolve()
DATA_DIR = Path("D:/proyectos/Caso_aprendizaje-Melanomas/data/raw")
SPLITS = ["train", "val", "test"]
ALLOWED_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR   = DATA_DIR / "val"
TEST_DIR  = DATA_DIR / "test"
QUAR_DIR = DATA_DIR / "_quarantine"
QUAR_DIR.mkdir(exist_ok=True)
for p in (TRAIN_DIR, VAL_DIR, TEST_DIR): print(p, "exists:", p.exists())

# Hiperparámetros
CLASSES = 2
BATCH   = 32
ROWS = COLS = 224
INPUT_CH = 3
EPOCHS  = 20
TEST_MAX_SAMPLES = 3000
SEED    = 42


D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\train exists: True
D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\val exists: True
D:\proyectos\Caso_aprendizaje-Melanomas\data\raw\test exists: True


## 📌 ¿Por qué separar en *train*, *validation* y *test*?

En proyectos de *Machine Learning* es fundamental dividir los datos en diferentes conjuntos:

- **Train (entrenamiento)**  
  Se utiliza para que el modelo aprenda los patrones de las imágenes.  

- **Validation (validación)**  
  Sirve para ajustar hiperparámetros (número de capas, tasa de aprendizaje, etc.) y comparar modelos sin mirar el *test*.  
  👉 Es una especie de "examen parcial": nos avisa si el modelo se está sobreajustando (*overfitting*) o si generaliza bien.  

- **Test (prueba final)**  
  Es el conjunto que se mantiene completamente aislado durante todo el desarrollo.  
  👉 Representa el "examen final" y nos da una estimación real del rendimiento del modelo en datos nunca vistos.  

✅ En este caso (clasificación de melanomas), separar en *train* y *val* es crucial porque:  
- Ayuda a evitar *overfitting*, que sería muy peligroso si el modelo solo memoriza imágenes.  
- Permite evaluar de manera más realista la capacidad de generalización antes de aplicarlo en un entorno clínico.  


In [14]:
# Función para contar imágenes por carpeta
def count_images_in_dir(dir_path: Path):
    if not dir_path.exists():
        return {}
    counts = {}
    for cls_dir in dir_path.iterdir():
        if cls_dir.is_dir():
            n_imgs = sum(1 for f in cls_dir.rglob("*") if f.suffix.lower() in ALLOWED_EXTS)
            counts[cls_dir.name] = n_imgs
    return counts

# Recolectar conteos
all_counts = {}
for split in SPLITS:
    split_dir = DATA_DIR / split
    all_counts[split] = count_images_in_dir(split_dir)

# Mostrar tabla
import pandas as pd

df = pd.DataFrame(all_counts).fillna(0).astype(int).T
df["Total"] = df.sum(axis=1)
display(df)



,Benign,Malignant,Total
train,5346,4752,10098
val,943,838,1781
test,1000,1000,2000


## ⚡ Uso de GPU en PyTorch

Para acelerar el entrenamiento de los modelos, es importante usar la **GPU** en lugar de la **CPU** siempre que esté disponible.  

Con PyTorch podemos:  
- Detectar automáticamente si el sistema tiene una GPU con CUDA.  
- Mover el **modelo** y los **tensores** al dispositivo adecuado (`cuda` o `cpu`).  
- Asegurarnos de que todo el flujo de entrenamiento (inputs, labels, modelo, pérdidas) se ejecute en la GPU.  

El siguiente código hace esta detección y muestra qué dispositivo se está utilizando.


In [15]:
import torch

print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# prueba rápida en GPU
x = torch.randn(4096, 4096, device=device)
y = x @ x
print("Tensor en:", y.device)


CUDA disponible: True
GPU: NVIDIA GeForce MX250
Tensor en: cuda:0


## 🖼️ Dimensionado consistente de imágenes

Para que el modelo funcione con cualquier imagen, todas deben tener **mismo tamaño** y **misma normalización**.

- Tamaño objetivo: `224x224` (estándar para muchas CNN).
- Estrategia:
  - **Entrenamiento**: redimensionar + (opcional) aumentos de datos ligeros.
  - **Validación/Test**: solo redimensionar (sin aumentos).
  - **Inferencia**: usar **exactamente** la misma trasformación que `val/test`.

> Nota: es preferible **transformar al vuelo** (con `DataLoader`) y **no sobrescribir** las imágenes en disco. Si quieres un *export* a `data/processed/`, al final tienes una celda opcional para guardar las redimensionadas.


## A) Limpieza rápida (corruptas + orientación + RGB)

In [16]:

def clean_folder(split_dir: Path):
    n_total=n_quar=0
    for p in split_dir.rglob("*"):
        if p.suffix.lower() not in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}:
            continue
        n_total += 1
        try:
            with Image.open(p) as im:
                im = ImageOps.exif_transpose(im)   # respeta la orientación EXIF
                if im.mode != "RGB":
                    im = im.convert("RGB")         # fuerza 3 canales
                # No se guarda en disco: solo validamos que abre bien
        except Exception as e:
            n_quar += 1
            shutil.move(str(p), QUAR_DIR / p.name)
    print(f"{split_dir.name}: {n_total} imgs, movidas a cuarentena: {n_quar}")

for split in ["train","val","test"]:
    d = DATA_DIR / split
    if d.exists():
        clean_folder(d)
print("✅ Limpieza básica terminada (sin modificar imágenes buenas).")


train: 10098 imgs, movidas a cuarentena: 0
val: 1781 imgs, movidas a cuarentena: 0
test: 2000 imgs, movidas a cuarentena: 0
✅ Limpieza básica terminada (sin modificar imágenes buenas).


## B) Transforms coherentes (letterbox + resize + tensor)

In [17]:

class PadToSquare:
    def __init__(self, fill=0): self.fill = fill
    def __call__(self, img):
        w, h = img.size
        s = max(w, h)
        pad_left  = (s - w) // 2
        pad_right = s - w - pad_left
        pad_top   = (s - h) // 2
        pad_bottom= s - h - pad_top
        return TF.pad(img, [pad_left, pad_top, pad_right, pad_bottom], fill=self.fill)

# >>> Si vas a usar TRANSFER LEARNING (recomendado):
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = transforms.Compose([
    PadToSquare(fill=0),
    transforms.Resize((ROWS, COLS)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.05,0.05)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

eval_tfms = transforms.Compose([
    PadToSquare(fill=0),
    transforms.Resize((ROWS, COLS)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


train_ds = datasets.ImageFolder(DATA_DIR / "train", transform=train_tfms)
val_ds   = datasets.ImageFolder(DATA_DIR / "val",   transform=eval_tfms) if (DATA_DIR / "val").exists() else None
test_ds  = datasets.ImageFolder(DATA_DIR / "test",  transform=eval_tfms) if (DATA_DIR / "test").exists() else None

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=0, pin_memory=(device.type=="cuda"))
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda")) if val_ds else None
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda")) if test_ds else None

xb, yb = next(iter(train_dl))
print("Batch:", xb.shape)  # [B, 3, 224, 224]


Batch: torch.Size([32, 3, 224, 224])


## C) Inferencia de una imagen (mismo pipeline que val/test)